In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# 학습시킬 텍스트 파일 경로 설정
import pandas as pd

file_path = '/content/drive/MyDrive/DLproject/dataset/curse_dataset.txt'

# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    corpus = [line.strip() for line in file]

!pip install soynlp

from soynlp.word import WordExtractor

# WordExtractor를 사용하여 단어 추출기 초기화
word_extractor = WordExtractor()

# 단어 추출기에 텍스트 파일의 문장들을 입력하여 단어 점수 계산
word_extractor.train(corpus)

# WordExtractor로부터 단어 점수를 활용하여 단어를 추출
word_scores = word_extractor.extract()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 2.4 MB/s eta 0:00:00
training was done. used memory 0.220 Gb
all cohesion probabilities was computed. # words = 3940
all branching entropies was computed # words = 5575
all accessor variety was computed # words = 5575


In [4]:

from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_scores.items()}
l_tokenizer = LTokenizer(scores=scores)


from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)


from soynlp.normalizer import *


In [5]:
# 정제된 텍스트를 저장할 리스트 초기화
cleaned_corpus = []
# 각 문장에 대해 이모티콘 정제 적용
for sentence in corpus:
    cleaned_sentence = emoticon_normalize(sentence, num_repeats=3)
    cleaned_corpus.append(cleaned_sentence)


In [6]:
# 정제된 문장을 "문장" column으로 하는 DataFrame 생성
df = pd.DataFrame({'문장': cleaned_corpus})

# DataFrame 출력
print(df.head(10))

                                                  문장
0                                        좌배 까는건 ㅇㅂ|1
1                     집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ|0
2       개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ이라고 말못해서 삐진거야 빨갱아|1
3                                      세탁이라고 봐도 된다|0
4                                애새끼가 초딩도 아니고 ㅋㅋㅋ |1
5  731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...
6                                          재앙이한건햇노|1
7                   글쓴이 와꾸 승리에 비하면 방사능 피폭 원숭이 일듯..|1
8                  마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나|1
9                  은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌|0


In [7]:
# '|'을 기준으로 문자열 분리하고, 0과 1을 라벨로 저장
df['라벨'] = df['문장'].str.split('|').str[1].str.strip()

# '\n' 제거
df['문장'] = df['문장'].str.replace('\n', '')

# 라벨 및 | 제거
df['문장'] = df['문장'].str.replace('|0', '')
df['문장'] = df['문장'].str.replace('|1', '')

# 결과 출력
print(df)


                                                     문장 라벨
0                                             좌배 까는건 ㅇㅂ  1
1                          집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ  0
2            개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ이라고 말못해서 삐진거야 빨갱아  1
3                                           세탁이라고 봐도 된다  0
4                                     애새끼가 초딩도 아니고 ㅋㅋㅋ   1
...                                                 ... ..
5820         좌우 헬파이어 3개씩 6개 장착에 아파치보다 약하지만 20mm 기관포 장착임  0
5821  세금 내놓으라고 데모질 중 ㅋㅋ첩, 도둑놈 새끼들이 대통령 해처먹으니까 나도 같이 ...  1
5822                                          너가 한 말 중에  0
5823                                            제갈대중 ㅇㅂ  0
5824                           우리나라교회는 악마들이모여 주뎅이 처벌리고   1

[5825 rows x 2 columns]


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


# 문장과 라벨 추출
X = df['문장'].tolist()
y = df['라벨'].tolist()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [9]:
# 토크나이저 초기화
tokenizer = Tokenizer()

# 문장을 토큰화하여 정수 시퀀스로 변환
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
vocab_size = len(tokenizer.word_index) + 1

# 시퀀스 패딩
max_sequence_length = 100  # 적절한 시퀀스 길이 선택
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# 변환된 데이터 출력
print("토큰화된 시퀀스:\n", X_padded)
print("라벨:\n", y)

print(len(X_padded))

print(vocab_size)

print(tokenizer.word_index)

토큰화된 시퀀스:
 [[    0     0     0 ...  4252  2036    49]
 [    0     0     0 ...     8  4257     2]
 [    0     0     0 ...  4262  4263   927]
 ...
 [    0     0     0 ...    43    94   627]
 [    0     0     0 ...     0 27155    49]
 [    0     0     0 ... 27157 27158 27159]]
라벨:
 [1 0 1 ... 0 0 1]
5825
27160
{'ㅋㅋㅋ': 1, 'ㅋㅋ': 2, '다': 3, '그냥': 4, '존나': 5, '진짜': 6, '왜': 7, '더': 8, '근데': 9, '그': 10, '저': 11, '좀': 12, '내가': 13, '난': 14, '이': 15, 'ㅁㅈㅎ': 16, '하고': 17, 'ㅋ': 18, '지금': 19, '많이': 20, '뭐': 21, '안': 22, '돈': 23, '잘': 24, '아': 25, '하는': 26, '수': 27, '씨발': 28, '시발': 29, '같은': 30, '너무': 31, '그럼': 32, '있는': 33, '나도': 34, '니가': 35, '그리고': 36, '내': 37, '또': 38, 'ㄹㅇ': 39, '와': 40, '니': 41, '없는게': 42, '한': 43, '기사글은': 44, '링크': 45, '아니라': 46, '이게': 47, '하면': 48, 'ㅇㅂ': 49, '보면': 50, '그게': 51, '이런': 52, '아님': 53, '이제': 54, '할': 55, '걍': 56, '그렇게': 57, '그래서': 58, 'ㅇㅇ': 59, '무슨': 60, '뭔': 61, '아니고': 62, '그런': 63, '저렇게': 64, '그래도': 65, '다른': 66, '나': 67, '없음': 68, '저런': 69, '저거': 70, '너': 71, '함

In [11]:
# 토큰화된 시퀀스를 다시 텍스트로 변환하는 함수 정의
def sequences_to_texts(tokenizer, sequences):
    reverse_word_index = {value: key for key, value in tokenizer.word_index.items()}
    texts = []
    for seq in sequences:
        texts.append(' '.join([reverse_word_index.get(i, '') for i in seq if i != 0]))
    return texts

In [12]:
# 3. 토큰화된 시퀀스를 다시 텍스트로 변환
original_texts = sequences_to_texts(tokenizer, X_padded)

print(tokenizer.word_index)
print(sequences[0])
print(original_texts[0])

{'ㅋㅋㅋ': 1, 'ㅋㅋ': 2, '다': 3, '그냥': 4, '존나': 5, '진짜': 6, '왜': 7, '더': 8, '근데': 9, '그': 10, '저': 11, '좀': 12, '내가': 13, '난': 14, '이': 15, 'ㅁㅈㅎ': 16, '하고': 17, 'ㅋ': 18, '지금': 19, '많이': 20, '뭐': 21, '안': 22, '돈': 23, '잘': 24, '아': 25, '하는': 26, '수': 27, '씨발': 28, '시발': 29, '같은': 30, '너무': 31, '그럼': 32, '있는': 33, '나도': 34, '니가': 35, '그리고': 36, '내': 37, '또': 38, 'ㄹㅇ': 39, '와': 40, '니': 41, '없는게': 42, '한': 43, '기사글은': 44, '링크': 45, '아니라': 46, '이게': 47, '하면': 48, 'ㅇㅂ': 49, '보면': 50, '그게': 51, '이런': 52, '아님': 53, '이제': 54, '할': 55, '걍': 56, '그렇게': 57, '그래서': 58, 'ㅇㅇ': 59, '무슨': 60, '뭔': 61, '아니고': 62, '그런': 63, '저렇게': 64, '그래도': 65, '다른': 66, '나': 67, '없음': 68, '저런': 69, '저거': 70, '너': 71, '함': 72, '거': 73, '바로': 74, '보고': 75, '아니': 76, '일본': 77, '사람': 78, 'ㅅㅂ': 79, '아니냐': 80, '이거': 81, '어떻게': 82, '있고': 83, '없는': 84, '있는데': 85, '솔직히': 86, '다시': 87, '못': 88, '있다': 89, '그거': 90, '제대로': 91, '이미': 92, '공부': 93, '말': 94, '정말': 95, '있음': 96, '누가': 97, '그걸': 98, '딱': 99, '나는': 100, '말이': 101, '병신': 102

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 27159
등장 빈도가 2번 이하인 희귀 단어의 수: 25124
단어 집합에서 희귀 단어의 비율: 92.50708788983394
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 61.72114863644573


In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 2037


In [13]:
from sklearn.model_selection import train_test_split

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


Training data shape: (4660, 100)
Testing data shape: (1165, 100)


In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2



# 데이터 로드 및 전처리
file_path = '/content/drive/MyDrive/DLproject/dataset/curse_dataset.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    corpus = [line.strip() for line in file]

df = pd.DataFrame({'문장': corpus})
df['라벨'] = df['문장'].str.split('|').str[1].str.strip()
df['문장'] = df['문장'].str.replace('\n', '').str.replace('|0', '').str.replace('|1', '')

X = df['문장'].tolist()
y = df['라벨'].tolist()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# 토크나이저 초기화 시 OOV 토큰 설정
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
vocab_size = len(tokenizer.word_index) + 1  # 단어 집합 크기 설정

max_sequence_length = 100
X_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [ ]:

# 모델 정의
model_01 = Sequential()
model_01.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model_01.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model_01.add(MaxPooling1D(pool_size=2))
model_01.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model_01.add(GlobalMaxPooling1D())
model_01.add(Dense(128, activation='relu'))
model_01.add(Dropout(0.5))
model_01.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_01.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model_01.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# 모델 평가
loss, accuracy = model_01.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# 예측값 생성
y_pred = (model_01.predict(X_test) > 0.5).astype("int32")

# 분류 리포트 출력
print(classification_report(y_test, y_pred))

# F1 스코어 계산 및 출력
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")



overfitting 문제를 해결하면 acc, F1 score를 더 올릴 수 있을것이라 판단
dropout 비율을 키우고, 커널 사이즈를 줄이고, 정규화 텀 추가 및 필터 사이즈도 줄인다..

In [ ]:

# 모델 정의
model_02 = Sequential()
model_02.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model_02.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model_02.add(MaxPooling1D(pool_size=2))
model_02.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model_02.add(GlobalMaxPooling1D())
model_02.add(Dense(128, activation='relu'))
model_02.add(Dropout(0.6))
model_02.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_02.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model_02.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# 모델 평가
loss, accuracy = model_02.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# 예측값 생성
y_pred = (model_02.predict(X_test) > 0.5).astype("int32")

# 분류 리포트 출력
print(classification_report(y_test, y_pred))

# F1 스코어 계산 및 출력
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

Epoch 1/10
59/59 [==============================] - 6s 73ms/step - loss: 1.1169 - accuracy: 0.6462 - val_loss: 0.8334 - val_accuracy: 0.6760
Epoch 2/10
59/59 [==============================] - 6s 106ms/step - loss: 0.7491 - accuracy: 0.6486 - val_loss: 0.6726 - val_accuracy: 0.6760
Epoch 3/10
59/59 [==============================] - 4s 70ms/step - loss: 0.6019 - accuracy: 0.7532 - val_loss: 0.6085 - val_accuracy: 0.7253
Epoch 4/10
59/59 [==============================] - 4s 68ms/step - loss: 0.3008 - accuracy: 0.9268 - val_loss: 0.7399 - val_accuracy: 0.6266
Epoch 5/10
59/59 [==============================] - 8s 136ms/step - loss: 0.1755 - accuracy: 0.9742 - val_loss: 0.6645 - val_accuracy: 0.7597
Epoch 6/10
59/59 [==============================] - 7s 124ms/step - loss: 0.1046 - accuracy: 0.9882 - val_loss: 0.7008 - val_accuracy: 0.7275
Epoch 7/10
59/59 [==============================] - 7s 125ms/step - loss: 0.0620 - accuracy: 0.9981 - val_loss: 0.7211 - val_accuracy: 0.7371
Epoch 8/1

02 모델에서 cnn 을 2개 -> 1개로 바꿔본다 => 기각

In [ ]:
from tensorflow.keras.regularizers import l2
# 모델 정의
model_03 = Sequential()
model_03.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model_03.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model_03.add(GlobalMaxPooling1D())
model_03.add(Dense(128, activation='relu'))
model_03.add(Dropout(0.6))
model_03.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_03.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model_03.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# 모델 평가
loss, accuracy = model_03.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# 예측값 생성
y_pred = (model_03.predict(X_test) > 0.5).astype("int32")

# 분류 리포트 출력
print(classification_report(y_test, y_pred))

# F1 스코어 계산 및 출력
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

Epoch 1/10
59/59 [==============================] - 6s 92ms/step - loss: 0.9182 - accuracy: 0.6414 - val_loss: 0.7233 - val_accuracy: 0.6760
Epoch 2/10
59/59 [==============================] - 4s 60ms/step - loss: 0.6796 - accuracy: 0.6483 - val_loss: 0.6323 - val_accuracy: 0.6760
Epoch 3/10
59/59 [==============================] - 3s 57ms/step - loss: 0.5107 - accuracy: 0.7690 - val_loss: 0.5820 - val_accuracy: 0.7586
Epoch 4/10
59/59 [==============================] - 4s 63ms/step - loss: 0.1609 - accuracy: 0.9828 - val_loss: 0.6266 - val_accuracy: 0.7511
Epoch 5/10
59/59 [==============================] - 5s 81ms/step - loss: 0.0678 - accuracy: 0.9981 - val_loss: 0.6418 - val_accuracy: 0.7554
Epoch 6/10
59/59 [==============================] - 4s 60ms/step - loss: 0.0414 - accuracy: 0.9995 - val_loss: 0.6315 - val_accuracy: 0.7414
Epoch 7/10
59/59 [==============================] - 3s 59ms/step - loss: 0.0314 - accuracy: 0.9997 - val_loss: 0.6423 - val_accuracy: 0.7457
Epoch 8/10
59

배치 크기 실험 32, 64, 128

In [ ]:

# 조기 종료 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 정의
model_03 = Sequential()
model_03.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model_03.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model_03.add(MaxPooling1D(pool_size=2))
model_03.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model_03.add(GlobalMaxPooling1D())
model_03.add(Dense(128, activation='relu'))
model_03.add(Dropout(0.6))
model_03.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_03.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 다양한 배치 크기 실험
for batch_size in [32, 64, 128]:
    print(f"Training with batch size: {batch_size}")

    # 모델 학습
    history = model_03.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

    # 모델 평가
    loss, accuracy = model_03.evaluate(X_test, y_test)
    print(f"Test Accuracy with batch size {batch_size}: {accuracy:.4f}")

    # 예측값 생성
    y_pred = (model_03.predict(X_test) > 0.5).astype("int32")

    # 분류 리포트 출력
    print(classification_report(y_test, y_pred))

    # F1 스코어 계산 및 출력
    f1 = f1_score(y_test, y_pred)
    print(f"F1 Score with batch size {batch_size}: {f1:.4f}")

Training with batch size: 32
Epoch 1/10
117/117 [==============================] - 38s 69ms/step - loss: 0.9409 - accuracy: 0.6451 - val_loss: 0.6817 - val_accuracy: 0.6760
Epoch 2/10
117/117 [==============================] - 6s 54ms/step - loss: 0.6608 - accuracy: 0.6532 - val_loss: 0.6089 - val_accuracy: 0.7210
Epoch 3/10
117/117 [==============================] - 7s 64ms/step - loss: 0.3967 - accuracy: 0.8806 - val_loss: 0.6299 - val_accuracy: 0.7253
Epoch 4/10
117/117 [==============================] - 7s 58ms/step - loss: 0.1714 - accuracy: 0.9681 - val_loss: 0.6628 - val_accuracy: 0.7371
Epoch 5/10
117/117 [==============================] - 7s 59ms/step - loss: 0.0932 - accuracy: 0.9893 - val_loss: 0.7155 - val_accuracy: 0.7242
Epoch 6/10
117/117 [==============================] - 7s 59ms/step - loss: 0.0593 - accuracy: 0.9976 - val_loss: 0.7750 - val_accuracy: 0.7039
Epoch 7/10
117/117 [==============================] - 7s 56ms/step - loss: 0.0464 - accuracy: 0.9984 - val_loss:

F1 score 해석 : 1에 대한 recall 값이 낮다
batch size는 64로 결정하고, 결정 임계값을 조정해봄

In [21]:
# 모델 정의
model_04 = Sequential()
model_04.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model_04.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model_04.add(GlobalMaxPooling1D())
model_04.add(Dense(128, activation='relu'))
model_04.add(Dropout(0.6))
model_04.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_04.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model_04.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# 모델 평가
loss, accuracy = model_04.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# 예측값 생성
y_pred = (model_04.predict(X_test) > 0.3).astype("int32")

# 분류 리포트 출력
print(classification_report(y_test, y_pred))

# F1 스코어 계산 및 출력
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

Epoch 1/10
59/59 [==============================] - 6s 72ms/step - loss: 0.9158 - accuracy: 0.6446 - val_loss: 0.7181 - val_accuracy: 0.6760
Epoch 2/10
59/59 [==============================] - 6s 99ms/step - loss: 0.6770 - accuracy: 0.6486 - val_loss: 0.6326 - val_accuracy: 0.6813
Epoch 3/10
59/59 [==============================] - 5s 81ms/step - loss: 0.4659 - accuracy: 0.8222 - val_loss: 0.5915 - val_accuracy: 0.7597
Epoch 4/10
59/59 [==============================] - 4s 70ms/step - loss: 0.1366 - accuracy: 0.9828 - val_loss: 0.6476 - val_accuracy: 0.7672
Epoch 5/10
59/59 [==============================] - 5s 92ms/step - loss: 0.0628 - accuracy: 0.9979 - val_loss: 0.6373 - val_accuracy: 0.7597
Epoch 6/10
59/59 [==============================] - 5s 83ms/step - loss: 0.0388 - accuracy: 0.9997 - val_loss: 0.6188 - val_accuracy: 0.7468
Epoch 7/10
59/59 [==============================] - 4s 68ms/step - loss: 0.0278 - accuracy: 1.0000 - val_loss: 0.6267 - val_accuracy: 0.7425
Epoch 8/10
59

모델 4를 최종 모델로 정의한다.

F1 Score가 높아짐을 확인함.
그러나 욕설 탐지 -> 제재 의 논리를 생각해 봤을 때,

양성으로 판단한 것들 중 양성인 비율인 precision이 높아야 함. 즉 억울한 피해를 받는 유저가 적어야 함

FN보다 FP가 적어야 한다 -> 임계값을 높여서 실험

In [ ]:
# 모델 정의
model_05 = Sequential()
model_05.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model_05.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model_05.add(GlobalMaxPooling1D())
model_05.add(Dense(128, activation='relu'))
model_05.add(Dropout(0.6))
model_05.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_05.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model_05.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# 모델 평가
loss, accuracy = model_05.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# 예측값 생성
y_pred = (model_05.predict(X_test) > 0.7).astype("int32")

# 분류 리포트 출력
print(classification_report(y_test, y_pred))

# F1 스코어 계산 및 출력
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

Epoch 1/10
59/59 [==============================] - 7s 95ms/step - loss: 0.9142 - accuracy: 0.6430 - val_loss: 0.7180 - val_accuracy: 0.6760
Epoch 2/10
59/59 [==============================] - 4s 66ms/step - loss: 0.6661 - accuracy: 0.6548 - val_loss: 0.6126 - val_accuracy: 0.7006
Epoch 3/10
59/59 [==============================] - 4s 60ms/step - loss: 0.3979 - accuracy: 0.8664 - val_loss: 0.5997 - val_accuracy: 0.7532
Epoch 4/10
59/59 [==============================] - 4s 74ms/step - loss: 0.1236 - accuracy: 0.9852 - val_loss: 0.6587 - val_accuracy: 0.7618
Epoch 5/10
59/59 [==============================] - 5s 79ms/step - loss: 0.0607 - accuracy: 0.9992 - val_loss: 0.6325 - val_accuracy: 0.7511
Epoch 6/10
59/59 [==============================] - 4s 63ms/step - loss: 0.0380 - accuracy: 0.9995 - val_loss: 0.6476 - val_accuracy: 0.7543
Epoch 7/10
59/59 [==============================] - 4s 67ms/step - loss: 0.0285 - accuracy: 1.0000 - val_loss: 0.6496 - val_accuracy: 0.7629
Epoch 8/10
59

# 여기서부터 LIME 알고리즘 코드

- model_05 대신 최종적으로 결과값 뽑은 모델 넣어서 알고리즘 적용하시면 됩니다.

In [14]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=340cc27f0c172bb4652ad0eaf2a1b213f110ff6e693390e3be84b287145c7bc5
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [15]:
# 라벨이 1인 데이터의 인덱스 찾기
positive_indices = [i for i, label in enumerate(y_test) if label == 1]

# 라벨이 1인 데이터 10개 출력
for idx in positive_indices[:10]:
    text_sequence = X_test[idx]
    text_str = " ".join([str(tokenizer.index_word[token]) for token in text_sequence if token != 0])
    print(f"Index: {idx}, Text: {text_str}, Label: {y_test[idx]}")


Index: 2, Text: 보증금 없다고 안주는 거지 주인 새끼들도 존나 많음돈 앞에선 다 거짓이고 눈탱이야 ㅋㅋ, Label: 1
Index: 9, Text: 남자들 성욕주체못하는 병신만들고 뭐만해도 처벌받게하려는거지, Label: 1
Index: 13, Text: 그 연장선에서 박사모틀딱을 다 죽여야 그게 보수가 사는 가장 중요한거임 ㅋㅋ 설명하자면 길어지는데 암튼 틀틀거리는거는 잘못 아님, Label: 1
Index: 15, Text: 요즘에는 대행업체에서 소방점검을 많이 하기 때문에 저런 일은 많이 줄었지만 그래도 장사 좀 해보려고 건물에 들어가서 시설 설비 갖추고나면 꼭 점검 한답시고 들이닥치는 소방관 새끼들이 있다두명도 아닌 꼭 한 명이 와가지고 존나 말도 안 되는 걸로 트집을 쳐 잡는다돈 달라는 소리지 씨발 새끼들근데 좆같은건 그렇게 한 명이 왔다간건 정식 점검이 아니라서 이후에 소방점검이 또 와존나 씨발 한국은 장사 한 번 해먹기도 좆같은 나라다, Label: 1
Index: 16, Text: 이래서 애새끼 함부로 싸지르는거 아니다, Label: 1
Index: 23, Text: 현금이 많으면 노인들 죽었을때 청소업체에서 꿀꺽하는돈도 많겠네 개꿀, Label: 1
Index: 24, Text: 특히 인터넷쇼핑 좆같은문자랑 번호 입력할 필요없이 지문한번 갖다대면 되는게 좋다, Label: 1
Index: 26, Text: 틀니, Label: 1
Index: 27, Text: 착취 박이라고 바꿔라 머 홍어만 채용하나 ㅎ, Label: 1
Index: 31, Text: 저열한 민족성 미국 이승만 박정희라는 변수가 있어서 본래 민족 모습을 잠시 벗어난거였지 원래 우리 민족 본 모습은 북괴임 다시 그길로 가는게 보이잖아, Label: 1


In [16]:
from lime.lime_text import LimeTextExplainer
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.font_manager as fm

In [27]:
# 예측할 텍스트 샘플 선택
idx = 2
sample_text = X_test[idx]

# 모델의 예측 확률 함수 정의
def predict_proba(texts):
    tokenized_texts = tokenizer.texts_to_sequences(texts)
    padded_texts = pad_sequences(tokenized_texts, maxlen=max_sequence_length, padding='post')
    return np.hstack((1 - model_04.predict(padded_texts), model_04.predict(padded_texts)))

# LIME 설명기 생성
explainer = LimeTextExplainer(class_names=["정상", "욕설"])

# LIME을 사용하여 예측 설명 생성
sample_text_str = [" ".join([str(tokenizer.index_word[token]) for token in seq if token != 0]) for seq in [X_test[idx]]]
exp = explainer.explain_instance(sample_text_str[0], predict_proba, num_features=6)

# 설명 출력
exp.show_in_notebook(text=True)

157/157 [==============================] - 2s 14ms/step
